# Spatial Filtering w/ LOMRs

Accessing LOMRs can now be done through ESRI's RESTful API. I now want to experiment with extracting all the floodplain delineations that exist inside of LOMR boundaries.

## Background

Part of FEMA's process for publishing floodplain updates to their GIS is cutting the floodplains with the LOMR boundary. In this way, there is a clean break in the floodplains, and spatial queries are greatly simplified: all we need to do in order to extract the changed floodplains is filter spatially:

```python
lomr_boundary.contains(floodplain_geometry)
## OR ##
floodplain_geometry.within(lomr_boundary)
```

In [ ]:
from arcgis.gis import GIS
from arcgis.features import Feature, FeatureLayer, FeatureLayerCollection
from arcgis.geometry import Geometry, filters, union, buffer

Extract Boulder city limits, FEMA lomr boundaries, and FEMA special flood hazard areas (sfha) polygons

In [ ]:
city_lims_url = "https://maps.bouldercolorado.gov/arcgis/rest/services/plan/CityLimits/MapServer/0"
city = FeatureLayer(city_lims_url)

nfhl_url = "https://hazards.fema.gov/gis/nfhl/rest/services/public/NFHL/MapServer"
nfhl = FeatureLayerCollection(nfhl_url)
lomr = nfhl.layers[1]
sfha = nfhl.layers[27]

Define the output spatial reference

In [ ]:
sr = 2876 # NAD83(HARN) / Colorado North (ftUS)

Extract city limits as a spatial filter object, and get all lomrs inside

In [ ]:
# using the unioned city boundary polygon
anon_gis = GIS()
city_lims = city.query(out_sr=sr)
city_geoms = [poly.geometry for poly in city_lims.features]
city_union = union(spatial_ref=sr, geometries=city_geoms, gis=anon_gis)
geom_filter = filters.intersects(city_union, sr=sr)

The most recent CRS audit required the city to report on LOMRs after August 16, 2018. So the result we want is all LOMRs on or after this date that reside within the city's limits.

In [ ]:
date_str = '2018-08-16'
clause = f"STATUS = 'Effective' AND EFF_DATE >= '{date_str}'"

In [ ]:
boulder_lomrs = lomr.query(where=clause,
                           geometry_filter=geom_filter,
                           out_sr=sr)

In [ ]:
boulder_lomrs.sdf

Now that we've collected all lomr geometries and attributes that have gone effective after the comparison date, let's see if we can union all the geometries and use the result to query the SFHA.

In [ ]:
lomr_geoms = [Geometry(lomr.geometry) for lomr in boulder_lomrs.features]
lomr_union = union(spatial_ref=sr, geometries=lomr_geoms, gis=anon_gis)
lomr_geom_filter = filters.contains(lomr_union, sr=sr)

In [ ]:
lomr_flood = sfha.query(geometry_filter=lomr_geom_filter,
                        out_fields=['FLD_AR_ID', 'STUDY_TYP', 'FLD_ZONE', 'ZONE_SUBTY', 'SFHA_TF', 'STATIC_BFE'],
                        distance=1,
                        units="esriSRUnit_Foot",
                        out_sr=sr)

In [ ]:
lomr_flood.sdf

In [ ]:
flood_geoms = [Geometry(f.geometry) for f in lomr_flood.features]
flood_union = union(spatial_ref=sr, geometries=flood_geoms, gis=anon_gis)
flood_union

In [ ]:
m = anon_gis.map('Boulder, CO', 12)
m.draw(shape=flood_union)
m